In [1]:

######## snakemake preamble start (automatically inserted, do not edit) ########
library(methods)
Snakemake <- setClass(
    "Snakemake",
    slots = c(
        input = "list",
        output = "list",
        params = "list",
        wildcards = "list",
        threads = "numeric",
        log = "list",
        resources = "list",
        config = "list",
        rule = "character",
        bench_iteration = "numeric",
        scriptdir = "character",
        source = "function"
    )
)
snakemake <- Snakemake(
    input = list('data/Methylation_predictions_train_20_predict_80/Methylation_pred_lasso_normalized_trimmed.rds', "input" = 'data/Methylation_predictions_train_20_predict_80/Methylation_pred_lasso_normalized_trimmed.rds'),
    output = list('data/Methylation_predictions_train_20_predict_80/Methylation_pred_lasso_normalized_trimmed_formatted.rds', "output_formatted_pred" = 'data/Methylation_predictions_train_20_predict_80/Methylation_pred_lasso_normalized_trimmed_formatted.rds'),
    params = list(),
    wildcards = list(),
    threads = 1,
    log = list('logs/processed_notebooks/Methylation_formatting_predictions.r.ipynb', "notebook" = 'logs/processed_notebooks/Methylation_formatting_predictions.r.ipynb'),
    resources = list('mem_mb', 'disk_mb', 'tmpdir', 'time', "mem_mb" = 200000, "disk_mb" = 1000, "tmpdir" = '/scratch/57589790', "time" = '20:00:00'),
    config = list(),
    rule = 'methylation_format',
    bench_iteration = as.numeric(NA),
    scriptdir = '/faststorage/project/DELFI1/Workspaces/CarmenAndAnika/prep_formatting_normalizing',
    source = function(...){
        wd <- getwd()
        setwd(snakemake@scriptdir)
        source(...)
        setwd(wd)
    }
)
setwd('/faststorage/project/DELFI1/Workspaces/CarmenAndAnika');

######## snakemake preamble end #########


# Format ATAC predictions into wider format

In [2]:
library(tidyverse)
install.packages("tidyr", repos='http://cran.us.r-project.org')
library(tidyr)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [3]:
input_file = snakemake@input[["input"]]
output_file = snakemake@output[["output_formatted_pred"]]

In [4]:

#pred_ATAC = read.csv( "../data/ATAC_predictions_on_all_samples/ATAC_pred_lasso_std_FALSE.csv")
#dim(pred_ATAC)

pred_ATAC = readRDS(input_file)
dim(pred_ATAC)

[1] 4248056       4

In [5]:
head(pred_ATAC)

sample,bin,ATAC_observed,ATAC_predicted
PGDX16568P,chr10_400,59.09021,50.10720
PGDX16568P,chr10_40,32.87098,38.43050
PGDX16568P,chr10_4,65.82263,50.49563
PGDX16568P,chr10_401,39.20313,36.26990
PGDX16568P,chr10_402,35.89876,37.08899
PGDX16568P,chr10_403,37.92318,35.73950


In [6]:
str(pred_ATAC)

tibble [4,248,056 × 4] (S3: tbl_df/tbl/data.frame)
 $ sample        : Factor w/ 473 levels "PGDX10344P1",..: 46 46 46 46 46 46 46 46 46 46 ...
 $ bin           : chr [1:4248056] "chr10_400" "chr10_40" "chr10_4" "chr10_401" ...
 $ ATAC_observed : num [1:4248056] 59.1 32.9 65.8 39.2 35.9 ...
 $ ATAC_predicted: num [1:4248056] 50.1 38.4 50.5 36.3 37.1 ...


In [7]:
pred_ATAC$sample <- as.character(pred_ATAC$sample)
pred_ATAC$bin <- as.character(pred_ATAC$bin)
pred_ATAC <- pred_ATAC %>% select(-ATAC_observed)

str(pred_ATAC)

tibble [4,248,056 × 3] (S3: tbl_df/tbl/data.frame)
 $ sample        : chr [1:4248056] "PGDX16568P" "PGDX16568P" "PGDX16568P" "PGDX16568P" ...
 $ bin           : chr [1:4248056] "chr10_400" "chr10_40" "chr10_4" "chr10_401" ...
 $ ATAC_predicted: num [1:4248056] 50.1 38.4 50.5 36.3 37.1 ...


In [8]:
#pred_ATAC <- pred_ATAC %>% select(-ATAC_observed)
pred_ATAC_wide <- pred_ATAC %>% tidyr::pivot_wider(names_from = bin, values_from = ATAC_predicted)
head(pred_ATAC_wide)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. 

sample,chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,...,chr9_39,chr9_391,chr9_392,chr9_393,chr9_394,chr9_395,chr9_396,chr9_397,chr9_398,chr9_399
PGDX16568P,50.10720,38.43050,50.49563,36.26990,37.08899,35.73950,42.94098,50.30837,50.66313,...,41.69085,48.46803,39.19122,40.33245,49.63915,42.58743,52.08225,51.84441,53.84734,35.06284
PGDX16569P,50.38967,43.69226,53.56724,40.19999,40.67496,45.66306,45.94454,53.63420,54.25514,...,39.99292,49.49178,46.14314,43.29620,50.04835,46.53519,51.45761,49.33736,63.15294,43.94147
PGDX16570P,54.09841,38.90010,57.60327,39.01121,42.97006,43.96420,46.19221,59.91990,57.36566,...,42.65936,49.98062,50.99560,44.71672,47.49022,51.49670,55.23129,52.10197,64.84029,47.99915
PGDX16571P,54.90257,40.47710,55.52637,41.71517,42.23254,43.34020,47.61174,58.27360,56.45344,...,48.34666,51.90264,55.36530,48.01901,52.85030,49.18811,59.65190,59.16895,64.22539,51.64244
PGDX16579P,54.40836,35.16049,57.19377,36.97275,38.29034,39.15255,47.17557,51.75797,52.33919,...,45.87922,49.45779,47.96223,40.21917,45.66219,48.05531,60.02450,58.92792,56.02913,43.20545
PGDX16580P,54.04343,40.79401,54.85352,36.91397,39.41826,41.45725,45.73760,56.09565,53.96090,...,43.38544,49.79330,50.40758,43.99041,51.16138,51.78069,57.08762,54.30238,64.64102,44.41425


In [9]:
dim(pred_ATAC_wide)

[1]   424 10020

In [10]:
saveRDS(pred_ATAC_wide, file = output_file)